In [22]:
import pandas as pd
import numpy as np

In [2]:
filepath = "/Users/thowe/MinersAI Dropbox/Tyler Howe/AK_sample_project/UNPROCESSED/agdb2-csv/agdb2/bestvalue.txt"

df = pd.read_csv(filepath, index_col=False)

df.columns = ["lab_id", "species", "units", "value", "method", "summary", "method"]

/var/folders/xc/wpkwk17x20z44scclf26ttt80000gn/T/ipykernel_12119/2591739854.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, index_col=False)


In [3]:
df

,lab_id,species,units,value,method,summary,method
0,62M110,Ag,ppm,-0.7,ES_SQ,"-0.7, ES_SQ",Ag_ppm_ES_SQ
1,F003691,Ag,ppm,2.0,ES_SQ,"2, ES_SQ; 2.3, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ
2,F003692,Ag,ppm,3.0,ES_SQ,"3, ES_SQ; 2.8, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ
3,F003696,Ag,ppm,0.5,ES_SQ,"0.5, ES_SQ; 0.8, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ
4,F003706,Ag,ppm,0.5,ES_SQ,"0.5, ES_SQ; 0.5, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ
...,...,...,...,...,...,...,...
7648838,M157366,Zr,ppm,22.0,WDX_Fuse,"22, WDX_Fuse; -35, NA",Zr_ppm_WDX_Fuse
7648839,M157367,Zr,ppm,41.0,WDX_Fuse,"41, WDX_Fuse; -68, NA",Zr_ppm_WDX_Fuse
7648840,M157368,Zr,ppm,60.0,WDX_Fuse,"60, WDX_Fuse; -89, NA",Zr_ppm_WDX_Fuse
7648841,M157369,Zr,ppm,41.0,WDX_Fuse,"41, WDX_Fuse; -39, NA",Zr_ppm_WDX_Fuse


In [ ]:
print(df['lab_id'].unique())

In [4]:
main_filepath = "/Users/thowe/MinersAI Dropbox/Tyler Howe/AK_sample_project/UNPROCESSED/agdb2-csv/agdb2/main.txt"
df_main = pd.read_csv(main_filepath, index_col=False)

/var/folders/xc/wpkwk17x20z44scclf26ttt80000gn/T/ipykernel_12119/2071819882.py:2: DtypeWarning: Columns (2,6,12,13,15,28,29,30,31,32,33,34,35,36,37,38,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_main = pd.read_csv(main_filepath, index_col=False)


In [5]:
print(df_main.columns)

print(df_main["longitude"].isna().mean() * 100)

Index(['lab_id', 'field_id', 'job_id', 'submitter', 'project', 'date_sub',
       'date_coll', 'country', 'state', 'quad', 'latitude', 'longitude',
       'spheroid', 'datum', 'loc_desc', 'depth', 'sample_src', 'method_col',
       'prim_class', 'sec_class', 'spec_name', 'comment', 'addl_attr',
       'geol_age', 'stratig', 'mineralzn', 'alteration', 'ign_form',
       'metamorph', 'facies_grd', 'src_rock', 'dep_env', 'zone', 'horizon',
       'saline', 'organics', 'ferritic', 'drainage', 'prep', 'mesh_size',
       'prv_job_id', 'prv_lab_id', 'Unnamed: 42'],
      dtype='object')
0.0


In [6]:
def calculate_overlap_percentage(
    df_a: pd.DataFrame, df_b: pd.DataFrame, column_name: str
) -> float:
    """
    Calculate the percentage of values in a specified column from DataFrame A that
    also exist in the same column of DataFrame B.

    :param df_a: First DataFrame (Dataset A)
    :param df_b: Second DataFrame (Dataset B)
    :param column_name: Name of the column to analyze
    :return: Percentage of values in df_a[column_name] found in df_b[column_name]
    """
    if column_name not in df_a.columns or column_name not in df_b.columns:
        raise ValueError(f"Column '{column_name}' must exist in both DataFrames.")

    values_a = set(df_a[column_name].dropna())
    values_b = set(df_b[column_name].dropna())

    overlap_count = len(values_a & values_b)
    total_count = len(values_a)

    return (overlap_count / total_count) * 100 if total_count > 0 else 0.0

In [7]:
percentage = calculate_overlap_percentage(df, df_main, 'lab_id')
print(f"Overlap percentage: {percentage:.2f}%")

Overlap percentage: 100.00%


In [8]:
def merge_lat_long(df_a: pd.DataFrame, df_b: pd.DataFrame) -> pd.DataFrame:
    """
    Merge latitude and longitude from df_B into df_A based on the 'lab_id' column.

    :param df_a: DataFrame A (without latitude/longitude)
    :param df_b: DataFrame B (with 'lab_id', 'latitude', and 'longitude')
    :return: Updated DataFrame A with latitude and longitude columns added
    """
    return df_a.merge(
        df_b[["lab_id", "latitude", "longitude"]], on="lab_id", how="left"
    )

In [9]:

df_merged = merge_lat_long(df, df_main)

In [10]:
df_merged

,lab_id,species,units,value,method,summary,method,latitude,longitude
0,62M110,Ag,ppm,-0.7,ES_SQ,"-0.7, ES_SQ",Ag_ppm_ES_SQ,59.55271,-154.49172
1,F003691,Ag,ppm,2.0,ES_SQ,"2, ES_SQ; 2.3, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ,55.63333,-160.65000
2,F003692,Ag,ppm,3.0,ES_SQ,"3, ES_SQ; 2.8, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ,55.63333,-160.65000
3,F003696,Ag,ppm,0.5,ES_SQ,"0.5, ES_SQ; 0.8, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ,55.63333,-160.65000
4,F003706,Ag,ppm,0.5,ES_SQ,"0.5, ES_SQ; 0.5, AA_F_AZ_H2O2_P",Ag_ppm_ES_SQ,55.63333,-160.65000
...,...,...,...,...,...,...,...,...,...
7648838,M157366,Zr,ppm,22.0,WDX_Fuse,"22, WDX_Fuse; -35, NA",Zr_ppm_WDX_Fuse,60.83333,-146.13333
7648839,M157367,Zr,ppm,41.0,WDX_Fuse,"41, WDX_Fuse; -68, NA",Zr_ppm_WDX_Fuse,60.26667,-147.66667
7648840,M157368,Zr,ppm,60.0,WDX_Fuse,"60, WDX_Fuse; -89, NA",Zr_ppm_WDX_Fuse,60.26667,-147.66667
7648841,M157369,Zr,ppm,41.0,WDX_Fuse,"41, WDX_Fuse; -39, NA",Zr_ppm_WDX_Fuse,60.26667,-147.66667


In [14]:
def reshape_geochem_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Reshape geochemical data so each row represents a unique lab_id and each species is a column.
    Only includes rows where the units are 'ppm' and retains latitude/longitude columns.

    :param df: Original geochemical DataFrame with columns ['lab_id', 'species', 'value', 'units', 'latitude', 'longitude']
    :return: Reshaped DataFrame with 'lab_id' as index and species as columns (all values in ppm), keeping latitude/longitude.
    """

    # Keep only rows where units are 'ppm'
    df_filtered = df[df["units"] == "ppm"].copy()

    # Ensure 'value' column is numeric
    df_filtered["value"] = pd.to_numeric(df_filtered["value"], errors="coerce")

    # Select only the relevant columns (lab_id, species, value, lat, long)
    df_subset = df_filtered[["lab_id", "species", "value"]]

    # Pivot the DataFrame so each species becomes a column
    df_pivot = df_subset.pivot_table(
        index="lab_id", columns="species", values="value", aggfunc="mean"
    )

    # Rename columns to include _ppm for clarity
    df_pivot.columns = [f"{col}_ppm" for col in df_pivot.columns]

    # Reset index so lab_id is a column again
    df_pivot = df_pivot.reset_index()

    # Retain latitude and longitude by merging them back into the reshaped DataFrame
    df_latlong = df[["lab_id", "latitude", "longitude"]].drop_duplicates(
        subset="lab_id"
    )
    df_final = df_pivot.merge(df_latlong, on="lab_id", how="left")

    return df_final

In [19]:

df_transformed = reshape_geochem_data(df_merged)
df_transformed.to_csv("geochem_reshaped.csv", index=False)
df_transformed

,lab_id,Ag_ppm,As_ppm,Au_ppm,B_ppm,Ba_ppm,Be_ppm,Bi_ppm,Cd_ppm,Ce_ppm,...,Tm_ppm,U_ppm,V_ppm,W_ppm,Y_ppm,Yb_ppm,Zn_ppm,Zr_ppm,latitude,longitude
0,62M110,-0.70,-200.0,NaN,-7.0,700.0,-1.0,-7.00,-50.00,-70.0,...,NaN,-500.0,150.0,-30.0,20.00,2.0,-100.0,100.0,59.55271,-154.49172
1,62M111,-0.70,-200.0,NaN,-7.0,700.0,-1.0,-7.00,-50.00,-70.0,...,NaN,-500.0,150.0,-30.0,20.00,3.0,-100.0,70.0,59.50931,-154.49273
2,62M112,-0.70,-200.0,NaN,-7.0,500.0,-1.0,-7.00,-50.00,-70.0,...,NaN,-500.0,200.0,-30.0,30.00,3.0,-100.0,150.0,59.50281,-154.47538
3,62M113,-0.70,-200.0,NaN,-7.0,300.0,-1.0,-7.00,-50.00,-70.0,...,NaN,-500.0,150.0,-30.0,30.00,3.0,-100.0,100.0,59.48241,-154.16356
4,63M1132,-0.70,-200.0,NaN,100.0,1000.0,-1.0,-7.00,-50.00,-70.0,...,NaN,-500.0,300.0,-30.0,20.00,2.0,-100.0,300.0,59.07693,-154.80478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256290,YL06788,0.02,5.8,0.0032,-1.0,44.4,0.3,0.17,0.04,12.7,...,NaN,0.5,50.0,0.1,4.16,NaN,22.1,3.5,59.80133,-155.30037
256291,YL06789,0.02,8.0,0.0013,1.0,53.2,0.3,0.16,0.05,14.3,...,NaN,0.5,54.0,0.1,4.49,NaN,28.0,2.8,59.80133,-155.30037
256292,YL06790,0.03,6.7,0.0014,-1.0,67.1,0.2,0.10,0.06,16.1,...,NaN,0.4,47.0,-0.1,5.28,NaN,29.9,3.4,59.80132,-155.29932
256293,YL06791,0.04,8.4,0.0016,-1.0,52.2,0.5,0.15,0.05,21.5,...,NaN,0.5,58.0,0.1,7.52,NaN,26.6,5.4,59.80133,-155.29823


In [23]:
df_transformed.columns

Index(['lab_id', 'Ag_ppm', 'As_ppm', 'Au_ppm', 'B_ppm', 'Ba_ppm', 'Be_ppm',
       'Bi_ppm', 'Cd_ppm', 'Ce_ppm', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm',
       'Dy_ppm', 'Er_ppm', 'Eu_ppm', 'Ga_ppm', 'Gd_ppm', 'Ge_ppm', 'Hf_ppm',
       'Hg_ppm', 'Hm_ppm', 'Ho_ppm', 'In_ppm', 'Ir_ppm', 'La_ppm', 'Li_ppm',
       'Lu_ppm', 'Mo_ppm', 'Nb_ppm', 'Nd_ppm', 'Ni_ppm', 'Os_ppm', 'Pb_ppm',
       'Pd_ppm', 'Pr_ppm', 'Pt_ppm', 'Rb_ppm', 'Re_ppm', 'Rh_ppm', 'Ru_ppm',
       'Sb_ppm', 'Sc_ppm', 'Se_ppm', 'Sm_ppm', 'Sn_ppm', 'Sr_ppm', 'Ta_ppm',
       'Tb_ppm', 'Te_ppm', 'Th_ppm', 'Tl_ppm', 'Tm_ppm', 'U_ppm', 'V_ppm',
       'W_ppm', 'Y_ppm', 'Yb_ppm', 'Zn_ppm', 'Zr_ppm', 'latitude',
       'longitude'],
      dtype='object')

In [27]:
def adjust_below_detection(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """
    Adjusts values below the detection limit in multiple specified columns.
    Any negative values are replaced with 1/3 of their absolute value.

    :param df: DataFrame containing the columns with possible below-DL values.
    :param columns: List of column names to adjust.
    :return: DataFrame with adjusted values.
    """
    df[columns] = df[columns].apply(
        lambda col: col.map(lambda x: abs(x) / 3 if x < 0 else x)
    )
    return df

In [28]:
element_columns = ['Ag_ppm', 'As_ppm', 'Au_ppm', 'B_ppm', 'Ba_ppm', 'Be_ppm',
       'Bi_ppm', 'Cd_ppm', 'Ce_ppm', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm',
       'Dy_ppm', 'Er_ppm', 'Eu_ppm', 'Ga_ppm', 'Gd_ppm', 'Ge_ppm', 'Hf_ppm',
       'Hg_ppm', 'Hm_ppm', 'Ho_ppm', 'In_ppm', 'Ir_ppm', 'La_ppm', 'Li_ppm',
       'Lu_ppm', 'Mo_ppm', 'Nb_ppm', 'Nd_ppm', 'Ni_ppm', 'Os_ppm', 'Pb_ppm',
       'Pd_ppm', 'Pr_ppm', 'Pt_ppm', 'Rb_ppm', 'Re_ppm', 'Rh_ppm', 'Ru_ppm',
       'Sb_ppm', 'Sc_ppm', 'Se_ppm', 'Sm_ppm', 'Sn_ppm', 'Sr_ppm', 'Ta_ppm',
       'Tb_ppm', 'Te_ppm', 'Th_ppm', 'Tl_ppm', 'Tm_ppm', 'U_ppm', 'V_ppm',
       'W_ppm', 'Y_ppm', 'Yb_ppm', 'Zn_ppm', 'Zr_ppm']

df_transformed_DLs = adjust_below_detection(df_transformed, element_columns)  # Apply to concentration column

In [29]:
export_filepath = "/Users/thowe/MinersAI Dropbox/Tyler Howe/AK_sample_project/PROCESSED/agdb_merged.csv"
df_transformed_DLs.to_csv(export_filepath, index=False)

In [30]:
df_transformed_DLs

,lab_id,Ag_ppm,As_ppm,Au_ppm,B_ppm,Ba_ppm,Be_ppm,Bi_ppm,Cd_ppm,Ce_ppm,...,Tm_ppm,U_ppm,V_ppm,W_ppm,Y_ppm,Yb_ppm,Zn_ppm,Zr_ppm,latitude,longitude
0,62M110,0.233333,66.666667,NaN,2.333333,700.0,0.333333,2.333333,16.666667,23.333333,...,NaN,166.666667,150.0,10.000000,20.00,2.0,33.333333,100.0,59.55271,-154.49172
1,62M111,0.233333,66.666667,NaN,2.333333,700.0,0.333333,2.333333,16.666667,23.333333,...,NaN,166.666667,150.0,10.000000,20.00,3.0,33.333333,70.0,59.50931,-154.49273
2,62M112,0.233333,66.666667,NaN,2.333333,500.0,0.333333,2.333333,16.666667,23.333333,...,NaN,166.666667,200.0,10.000000,30.00,3.0,33.333333,150.0,59.50281,-154.47538
3,62M113,0.233333,66.666667,NaN,2.333333,300.0,0.333333,2.333333,16.666667,23.333333,...,NaN,166.666667,150.0,10.000000,30.00,3.0,33.333333,100.0,59.48241,-154.16356
4,63M1132,0.233333,66.666667,NaN,100.000000,1000.0,0.333333,2.333333,16.666667,23.333333,...,NaN,166.666667,300.0,10.000000,20.00,2.0,33.333333,300.0,59.07693,-154.80478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256290,YL06788,0.020000,5.800000,0.0032,0.333333,44.4,0.300000,0.170000,0.040000,12.700000,...,NaN,0.500000,50.0,0.100000,4.16,NaN,22.100000,3.5,59.80133,-155.30037
256291,YL06789,0.020000,8.000000,0.0013,1.000000,53.2,0.300000,0.160000,0.050000,14.300000,...,NaN,0.500000,54.0,0.100000,4.49,NaN,28.000000,2.8,59.80133,-155.30037
256292,YL06790,0.030000,6.700000,0.0014,0.333333,67.1,0.200000,0.100000,0.060000,16.100000,...,NaN,0.400000,47.0,0.033333,5.28,NaN,29.900000,3.4,59.80132,-155.29932
256293,YL06791,0.040000,8.400000,0.0016,0.333333,52.2,0.500000,0.150000,0.050000,21.500000,...,NaN,0.500000,58.0,0.100000,7.52,NaN,26.600000,5.4,59.80133,-155.29823
